In [0]:
pip install lightgbm

In [0]:
pip install hyperopt

In [0]:
%pip install databricks-feature_engineering

In [0]:
 %restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import os
from datetime import datetime, date 
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
import numpy as np
from mlflow.models.signature import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
from mlflow.tracking import MlflowClient
import requests
from sklearn.metrics import accuracy_score

in this notebbok we will load fetaures from feature store create training set with it





train the model with hyperopt experiments with 30 trials 


logged the best parameter all other trials as child run under it
final save the model with best parameter registered in unity catalog

### creating training dataset

In [0]:
# we loading the delta table that we had saved in preprocessing notebook
base_4 = spark.table("ispl_databricks.model_logs.bd_500_features_sample_training").toPandas()

In [0]:
base_4.head()

In [0]:
base_4['type'].value_counts()

creating label dataframe consist of target and loan_id it will used fetch features for corresponding laon_id

In [0]:

# we are creating spark dataframe loan_id,target to create a label dataframe for creating training data set using feautre store
spark_label =  spark.createDataFrame(base_4[['loan_id','target']])


In [0]:
# Split the dataset into training and validation sets based on the 'type' column
input_final_df_train=base_4[base_4['type']=='training']
live_input_final_df=base_4[base_4['type']=='validation']

# Create a standardized label column ('actual_label') for model training and evaluation
# by copying values from the existing target column
input_final_df_train["actual_label"] = input_final_df_train["target"]
live_input_final_df["actual_label"] = live_input_final_df["target"]

In [0]:
input_final_df_train['actual_label'].value_counts(normalize=True)

In [0]:
live_input_final_df['actual_label'].value_counts(normalize=True)

In [0]:
live_input_final_df.shape

creating (loan_id,target) dataset for training and test 

In [0]:
#loading train and test data
train_data = base_4[base_4['type'] == 'training'].copy()
test_data = base_4[base_4['type'] == 'validation'].copy()


In [0]:
#creating label dataframe for training and test data
train_data = train_data[['target','loan_id']]
test_data = test_data[['target','loan_id']]


In [0]:
# creating a spark label dataframe for spark_train_label,spark_test_label
spark_train_label = spark.createDataFrame(train_data)
spark_test_label = spark.createDataFrame(test_data)

### creating train and test dataset using features store with the help of (loan_id,target) data frame corresponding to them

In [0]:

fe = FeatureEngineeringClient()

In [0]:
#loading features for training data using spark training label dataframe and loan_id
training_set = fe.create_training_set(
    df=spark_train_label,
    feature_lookups=[
        FeatureLookup(
            table_name="ispl_databricks.model_logs.bd_500_features_store_training",
            lookup_key="loan_id"
        )
    ],
    label="target"
)

In [0]:
#creating x_train and y_train
train_pd = training_set.load_df().toPandas()
X_train = train_pd.drop(['loan_id','target'], axis=1)
y_train = train_pd['target']

In [0]:

#loading features for test data using spark test label dataframe and loan_id
test_set = fe.create_training_set(
    df=spark_test_label,
    feature_lookups=[
        FeatureLookup(
            table_name="ispl_databricks.model_logs.bd_500_features_store_training",
            lookup_key="loan_id"
        )
    ],
    label="target"
)

In [0]:
#creating X_test,y_test
test_pd = test_set.load_df().toPandas()
X_test = test_pd.drop(['loan_id','target'], axis=1)
y_test = test_pd['target']

In [0]:
X_test.shape

In [0]:
X_train.shape

creating folder save models ,features and columns

In [0]:
model_trainYN = 1
data_version = "base"

input_dir = "/Volumes/ispl_databricks/default/training/ff_bd/input_dir"
output_dir = "/Volumes/ispl_databricks/default/training/ff_bd/output_dir_same_columns"
model_dir = "/Volumes/ispl_databricks/default/training/ff_bd/model_dir"

# Set data version
data_version = os.path.join("base")   # change here
# data_version = os.path.join("data_v5_new/top_20")   # alternative version

# Version-specific directories
input_dir_version = os.path.join(input_dir, data_version)
output_dir_version = os.path.join(output_dir, data_version)
model_dir_version = os.path.join(model_dir, data_version)

# Create directories if they do not exist
for directory in [output_dir_version, model_dir_version]:
    os.makedirs(directory, exist_ok=True)

# Model and feature file names
model_file_name = "lgb_model.pickle"
feature_file_name = "model_input_feature.pickle"


### hyperopt experiments

In [0]:

# Set MLflow experiment
mlflow.set_experiment("/Workspace/Shared/ff_bd/FF_bd_lgbm_all_columns_experiment")

# Train or load LGBMClassifier
if model_trainYN == 1:
    with mlflow.start_run(run_name=f"LGBM_{data_version}") as run:

        # -----------------------------
        # Hyperparameter tuning section
        # -----------------------------
        search_space = {
            'num_leaves': scope.int(hp.quniform('num_leaves', 20, 150, 1)),
            'max_depth': scope.int(hp.quniform('max_depth', 3, 15, 1)),
            'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
            'n_estimators': scope.int(hp.quniform('n_estimators', 100, 800, 50)),
            'min_child_samples': scope.int(hp.quniform('min_child_samples', 10, 100, 5)),
            'subsample': hp.uniform('subsample', 0.6, 1.0),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0)
        }

        def objective(params):
            # logging all trials under parent trial that is best parameter trial ith trial_id
            trial_id = len(trials.trials)
            with mlflow.start_run(run_name=f"trial_{trial_id}_LGBM",nested=True):
                model = lgb.LGBMClassifier(
                random_state=42,
                class_weight='balanced',
                **params
            )
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_test, y_test)],
                    eval_metric="binary_logloss"
            )
                val_pred_proba = model.predict_proba(X_test)[:, 1]
                val_logloss = -((y_test * np.log(val_pred_proba) + (1 - y_test) * np.log(1 - val_pred_proba)).mean())
                acccuracy = accuracy_score(y_test, model.predict(X_test))
                mlflow.log_metric("val_logloss", val_logloss)
                mlflow.log_metric("accuracy", acccuracy)   
                mlflow.log_params(params) 
                return {'loss': val_logloss, 'status': STATUS_OK}

        trials = Trials()
        best_params = fmin(
            fn=objective,
            space=search_space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials,
            rstate=np.random.default_rng(42)
        )

        # Convert integer-like floats back to ints
        best_params = {
            k: int(v) if isinstance(v, float) and v.is_integer() else v
            for k, v in best_params.items()
        }

        mlflow.log_params(best_params)
        mlflow.log_param("class_weight", "balanced")
        mlflow.log_param("random_state", 42)


        # -----------------------------
        # Train final model with best params
        # -----------------------------
        model = lgb.LGBMClassifier(
            random_state=42,
            class_weight='balanced',
            **best_params
        )

        model = model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            eval_names=["train", "valid"],
            eval_metric=["binary_logloss"]
        )

        # Infer model signature from training data and predictions
    

        # Save the trained model locally
        pickle.dump(
            model,
            open(os.path.join(output_dir_version, model_file_name), "wb")
        )
        joblib.dump(model, os.path.join(output_dir_version, "job_model.pkl"))

        # Save input feature list
        input_feature_model = X_train.columns.tolist()
        pickle.dump(
            input_feature_model,
            open(os.path.join(output_dir_version, feature_file_name), "wb")
        )
        mlflow.log_artifact(os.path.join(output_dir_version, feature_file_name))

        # Log some metrics (example)
        y_pred = model.predict(X_test)
        accuracy = (y_pred == y_test).mean()
        mlflow.log_metric("accuracy", accuracy)

else:
    # Load pre-trained model
    model = pickle.load(open(os.path.join(output_dir_version, model_file_name), "rb"))
    model = joblib.load(os.path.join(output_dir_version, "job_model.pkl"))

    # Load input feature list
    input_feature_model = pickle.load(
        open(os.path.join(output_dir_version, feature_file_name), "rb")
    )


### saving model

In [0]:
# dumping model at specific path to make it reusable
pickle.dump(
            model,
            open(os.path.join(output_dir_version, model_file_name), "wb")
        )

In [0]:
# loading the model that  save 
model = pickle.load(open(os.path.join(output_dir_version, model_file_name), "rb"))

In [0]:
#dumping model into model artifacts folder so that it can be easily in unity catalog
joblib.dump(model,'/Workspace/Shared/ff_bd/model_artifacts/500features.pkl')

In [0]:
#loading model
model =  joblib.load('/Workspace/Shared/ff_bd/model_artifacts/500features.pkl')

## ### model registary

### defining signature

In [0]:
#getting features used in model while training or the features on which model trained this prevents the schema mismatch while loggong into mlflow or while inferencing
training_features = model.feature_name_

In [0]:
# defing an input_x as a dataframe for example or defining signature
input_x = X_train[training_features].iloc[[1]]

In [0]:
# input_y for signature of output
input_y  = model.predict(input_x)


In [0]:
# defining  signature  as it required while logging in mlflow as it prevent schema mismatch
signature = infer_signature(input_x, input_y)

creating custom wrapper class to simplyfy registary and inferencing

In [0]:
#This custom MLflow PyFunc wrapper loads the trained model and its feature list, ensures schema consistency at inference time, and returns probability predictions for downstream evaluation or deployment it make easy to log the model into ml flow prevents schema mismatch.
class mlwrapper(mlflow.pyfunc.PythonModel):
    def load_context(self,context):
        #loading model that we had saved in artifacts
        self.model = joblib.load(context.artifacts['model_artifacts']+'/500features.pkl')
        #loading features used in model
        self.fc = model.feature_name_
        print(self.fc)
        
    def predict(self,context,model_input):
        df = model_input[self.fc]
        return self.model.predict_proba(df)

registering model into unity catalog wit mlflow experiment also logging accuracy of model

In [0]:
# Log the trained model to Unity Catalog using MLflow Model Registry
# Catalog  : ispl_databricks
# Schema   : model_logs
# Model    : ffbd_lgbm_all_columns_endpoint
# This enables centralized model governance, versioning, and deployment
with mlflow.start_run():
    # Log evaluation metric for model performance tracking
    mlflow.log_metric("test_accuracy", accuracy)

    
    # Log the trained model using a custom MLflow PyFunc wrapper
    # The wrapper ensures consistent feature selection and inference logic
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=mlwrapper(),
        # Path to model artifacts (e.g., serialized model files)
        artifacts={"model_artifacts": "/Workspace/Shared/ff_bd/model_artifacts"},
        # Register the model in Unity Catalog for lifecycle management
        registered_model_name="ispl_databricks.model_logs.ffbd_lgbm_all_columns_endpoint",
        # Attach model signature to enforce input/output schema validation
        signature=signature
    )

In [0]:
len(training_features)

In [0]:
lgb.plot_importance(model,max_num_features=30)

### training evaluation
lgb.plot_metric(model)

### inferencing

In [0]:
#model_name
model_name_500 = 'ispl_databricks.model_logs.ffbd_lgbm_all_columns_endpoint'

In [0]:
#loading versions of model
client =MlflowClient()
version500 = client.search_model_versions(f"name='{model_name_500}'")

In [0]:
# fetching latest version of model
latest_version500 = str(
    sorted(
        [int(versions.version) for versions in version500],
        reverse=True
    )[0]
)

In [0]:
#defining model_uri for latest model
model_uri = f"models:/ispl_databricks.model_logs.ffbd_lgbm_all_columns_endpoint/{latest_version500}"

In [0]:
#loading latest model
model = mlflow.pyfunc.load_model(model_uri)

In [0]:
# doing prediction on input_x
model.predict(input_x)

In [0]:
!pip freeze